In [1]:
#!/usr/bin/env python
# coding: utf-8

#%%
from rnaglib.tasks import GMSM 
from rnaglib.representations import GraphRepresentation
from rnaglib.data_loading import Collater
import torch
#from torch_geometric.loader import DataLoader
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv, GraphConv, SAGEConv, RGCNConv, global_mean_pool
import torch.optim as optim
import wandb
from collections import Counter
from torch.nn import BatchNorm1d, Dropout
import shutil
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef
from pathlib import Path

In [2]:
# this prevents the loading issue from occurring that removes x and y.
if Path('gmsm').exists():
    shutil.rmtree('gmsm')

ta = GMSM('gmsm')
ta.dataset.add_representation(GraphRepresentation(framework = 'pyg'))

Database was found and not overwritten
>>> Computing splits...
>>> Saving dataset.
>>> Done


In [6]:
train_ind, val_ind, test_ind = ta.split()
train_set = ta.dataset.subset(train_ind)
val_set = ta.dataset.subset(val_ind)
test_set = ta.dataset.subset(test_ind)

>>> Loading splits...


In [34]:
## This works
for i, batch in enumerate(train_set):
    print('.. ' + str(i))
    print(batch['graph'])

.. 0
Data(x=[10, 4], edge_index=[2, 24], edge_attr=[24], y=[10, 43])
.. 1
Data(x=[21, 4], edge_index=[2, 46], edge_attr=[46], y=[21, 43])
.. 2
Data(x=[6, 4], edge_index=[2, 10], edge_attr=[10], y=[6, 43])
.. 3
Data(x=[9, 4], edge_index=[2, 14], edge_attr=[14], y=[9, 43])
.. 4
Data(x=[36, 4], edge_index=[2, 92], edge_attr=[92], y=[36, 43])
.. 5
Data(x=[134, 4], edge_index=[2, 308], edge_attr=[308], y=[134, 43])
.. 6
Data(x=[48, 4], edge_index=[2, 116], edge_attr=[116], y=[48, 43])
.. 7
Data(x=[6, 4], edge_index=[2, 12], edge_attr=[12], y=[6, 43])
.. 8
Data(x=[139, 4], edge_index=[2, 342], edge_attr=[342], y=[139, 43])
.. 9
Data(x=[59, 4], edge_index=[2, 156], edge_attr=[156], y=[59, 43])
.. 10
Data(x=[221, 4], edge_index=[2, 526], edge_attr=[526], y=[221, 43])
.. 11
Data(x=[11, 4], edge_index=[2, 18], edge_attr=[18], y=[11, 43])
.. 12
Data(x=[45, 4], edge_index=[2, 110], edge_attr=[110], y=[45, 43])
.. 13
Data(x=[27, 4], edge_index=[2, 70], edge_attr=[70], y=[27, 43])
.. 14
Data(x=[20, 

In [21]:
collater = Collater(train_set)
train_loader = DataLoader(train_set, batch_size=1, shuffle=True, collate_fn=collater)
val_loader = DataLoader(val_set, batch_size=2, shuffle=False, collate_fn=collater)
test_loader = DataLoader(test_set, batch_size=2, shuffle=False, collate_fn=collater)

In [33]:
## This does not work
# I don't see the relation between the dimension of y and iterating over the train_loader

for i, batch in enumerate(train_loader):
    print('.. ' + str(i))
    print(batch)

'''
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[33], line 1
----> 1 for i, batch in enumerate(train_loader):
      2     print('.. ' + str(i))
      3     print(batch)

File /fs/gpfs41/lv11/fileset01/pool/pool-wyss/mamba/envs/RNA/lib/python3.12/site-packages/torch/utils/data/dataloader.py:631, in _BaseDataLoaderIter.__next__(self)
    628 if self._sampler_iter is None:
    629     # TODO(https://github.com/pytorch/pytorch/issues/76750)
    630     self._reset()  # type: ignore[call-arg]
--> 631 data = self._next_data()
    632 self._num_yielded += 1
    633 if self._dataset_kind == _DatasetKind.Iterable and \
    634         self._IterableDataset_len_called is not None and \
    635         self._num_yielded > self._IterableDataset_len_called:

File /fs/gpfs41/lv11/fileset01/pool/pool-wyss/mamba/envs/RNA/lib/python3.12/site-packages/torch/utils/data/dataloader.py:675, in _SingleProcessDataLoaderIter._next_data(self)
    673 def _next_data(self):
    674     index = self._next_index()  # may raise StopIteration
--> 675     data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
    676     if self._pin_memory:
    677         data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)

File /fs/gpfs41/lv11/fileset01/pool/pool-wyss/mamba/envs/RNA/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py:51, in _MapDatasetFetcher.fetch(self, possibly_batched_index)
     49         data = self.dataset.__getitems__(possibly_batched_index)
     50     else:
---> 51         data = [self.dataset[idx] for idx in possibly_batched_index]
     52 else:
     53     data = self.dataset[possibly_batched_index]

File /fs/gpfs41/lv11/fileset01/pool/pool-wyss/RNA/rnaglib/rnaglib/data_loading/rna_dataset.py:161, in RNADataset.__getitem__(self, idx)
    158 rna_graph = self.rnas[idx]
    160 rna_dict = {'rna': rna_graph}
--> 161 features_dict = self.compute_features(rna_dict)
    162 # apply representations to the res_dict
    163 # each is a callable that updates the res_dict
    164 for rep in self.representations:

File /fs/gpfs41/lv11/fileset01/pool/pool-wyss/RNA/rnaglib/rnaglib/data_loading/rna_dataset.py:254, in RNADataset.compute_features(self, rna_dict)
    252     features_dict['nt_features'] = feature_encoding
    253 if len(self.node_target_parser) > 0:
--> 254     target_encoding = self.get_nt_encoding(graph, encode_feature=False)
    255     features_dict['nt_targets'] = target_encoding
    256 return features_dict

File /fs/gpfs41/lv11/fileset01/pool/pool-wyss/RNA/rnaglib/rnaglib/data_loading/rna_dataset.py:216, in RNADataset.get_nt_encoding(self, g, encode_feature)
    214 try:
    215     node_feature = attrs[feature]
--> 216     node_feature_encoding = feature_encoder.encode(node_feature)
    217 except KeyError:
    218     node_feature_encoding = feature_encoder.encode_default()

File /fs/gpfs41/lv11/fileset01/pool/pool-wyss/RNA/rnaglib/rnaglib/utils/feature_maps.py:31, in OneHotEncoder.encode(self, value)
     29 try:
     30     ind = self.mapping[value]
---> 31     x[ind] = 1.
     32     return x
     33 except KeyError:

IndexError: index 43 is out of bounds for dimension 0 with size 43
'''

.. 0
{'graph': DataBatch(x=[14, 4], edge_index=[2, 32], edge_attr=[32], y=[14, 43], batch=[14], ptr=[2]), 'rna': [<networkx.classes.digraph.DiGraph object at 0x152e0ef02300>]}
.. 1
{'graph': DataBatch(x=[49, 4], edge_index=[2, 120], edge_attr=[120], y=[49, 43], batch=[49], ptr=[2]), 'rna': [<networkx.classes.digraph.DiGraph object at 0x152e094bcb90>]}
.. 2
{'graph': DataBatch(x=[41, 4], edge_index=[2, 104], edge_attr=[104], y=[41, 43], batch=[41], ptr=[2]), 'rna': [<networkx.classes.digraph.DiGraph object at 0x152e12e90c80>]}
.. 3
{'graph': DataBatch(x=[21, 4], edge_index=[2, 48], edge_attr=[48], y=[21, 43], batch=[21], ptr=[2]), 'rna': [<networkx.classes.digraph.DiGraph object at 0x152e0773c3b0>]}
.. 4
{'graph': DataBatch(x=[6, 4], edge_index=[2, 10], edge_attr=[10], y=[6, 43], batch=[6], ptr=[2]), 'rna': [<networkx.classes.digraph.DiGraph object at 0x152e05c60fe0>]}
.. 5
{'graph': DataBatch(x=[9, 4], edge_index=[2, 14], edge_attr=[14], y=[9, 43], batch=[9], ptr=[2]), 'rna': [<network

IndexError: index 43 is out of bounds for dimension 0 with size 43

----------------

In [ ]:

for batch in train_loader:
    print(batch)
    graph = batch['graph']
    print(f'Batch node features shape: \t{graph.x.shape}')
    print(f'Batch edge index shape: \t{graph.edge_index.shape}')
    print(f'Batch labels shape: \t\t{graph.y.shape}')
    break

# Printing some statistics

def calculate_length_statistics(loader):
    lengths = [data['graph'].x.shape[0] for data in loader.dataset]
    
    max_length = np.max(lengths)
    min_length = np.min(lengths)
    avg_length = np.mean(lengths)
    median_length = np.median(lengths)
    
    return {
        "max_length": max_length,
        "min_length": min_length,
        "average_length": avg_length,
        "median_length": median_length
    }

stats = calculate_length_statistics(train_loader)
print("Max Length:", stats["max_length"])
print("Min Length:", stats["min_length"])
print("Average Length:", stats["average_length"])
print("Median Length:", stats["median_length"])


def calculate_fraction_of_ones(loader):
    total_ones = 0
    total_elements = 0
    
    for batch in loader.dataset:
        y = batch['graph'].y  
        total_ones += (y == 1).sum().item()
        total_elements += y.numel()
    
    fraction_of_ones = total_ones / total_elements if total_elements > 0 else 0
    return fraction_of_ones

fraction = calculate_fraction_of_ones(train_loader)
print("Fraction of positives:", fraction)

def count_unique_attrs(train_loader):
    unique_edge_attrs = set()
    unique_node_attrs = set()
    
    for batch in train_loader.dataset:
        unique_edge_attrs.update(batch['graph'].edge_attr.tolist())

    return len(unique_edge_attrs), unique_edge_attrs

num_unique_edge_attrs, unique_edge_attrs = count_unique_attrs(train_loader)
print("Number of unique edge attributes:", num_unique_edge_attrs)
print("Unique edge attributes:", unique_edge_attrs)


def get_graph_labels(graph_y, batch):
    graph_labels = []
    for i in range(batch.max().item() + 1):
        node_indices = (batch == i).nonzero(as_tuple=True)[0]
        graph_labels.append(graph_y[node_indices[0]].argmax().item())
    return torch.tensor(graph_labels, dtype=torch.long, device=device)

# Model
#config
project_name = "gmsm_task_eval"
learning_rate = 0.0001
epochs = 500
batch_size = 1
dropout_rate = 0.1
num_layers = 2
batch_norm = True

wandb.init(project=project_name, config={
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size,
    "dropout_rate": dropout_rate,  
    "num_layers": num_layers, 
    "batch_norm": batch_norm, 
})

class RGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes, num_unique_edge_attrs):
        super(RGCN, self).__init__()
        self.conv1 = RGCNConv(num_node_features, 16, num_unique_edge_attrs)
        self.bn1 = BatchNorm1d(16)  
        self.dropout1 = Dropout(0.1) 
        self.conv2 = RGCNConv(16, 32, num_unique_edge_attrs)
        self.bn2 = BatchNorm1d(32)
        self.dropout2 = Dropout(0.1) 
        self.fc = Linear(32, num_classes)


    def forward(self, data):
        x, edge_index, edge_type = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_type)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.conv2(x, edge_index, edge_type)
        x = self.bn2(x)
        x = self.dropout2(x)
        x = global_mean_pool(x, data.batch) 
        x = self.fc(x)
        return x


num_classes = train_set[0]['graph'].y.shape[1]
model = RGCN(train_set.input_dim, num_classes, num_unique_edge_attrs)


# Training

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.NLLLoss()


# Evaluation function to get predictions and calculate metrics
def get_predictions_and_loss(loader):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0
    
    for batch in loader:
        graph = batch['graph']
        graph = graph.to(device)
        out = model(graph)
        labels = get_graph_labels(graph.y, graph.batch)  # Convert node-level labels to graph-level labels
        loss = criterion(out, labels)
        total_loss += loss.item()
        preds = out.argmax(dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(graph.y.tolist()) 

    avg_loss = total_loss / len(loader)
    return all_preds, all_labels, avg_loss

def calculate_metrics(loader):
    preds, labels, avg_loss = get_predictions_and_loss(loader)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    mcc = matthews_corrcoef(labels, preds)
    return accuracy, f1, auc, avg_loss, mcc  

# Training function
def train():
    model.train()
    for batch in train_loader:
        graph = batch['graph']
        graph = graph.to(device)
        optimizer.zero_grad()
        out = model(graph)
        labels = get_graph_labels(graph.y, graph.batch)  # Convert node-level labels to graph-level labels
        loss = criterion(out, labels)        
        loss.backward()
        optimizer.step()
        wandb.log({"train_loss": loss.item()})

# Main training loop
for epoch in range(5000):
    train()
    train_acc, train_f1, train_auc, train_loss, train_mcc = calculate_metrics(train_loader) 
    val_acc, val_f1, val_auc, val_loss, val_mcc = calculate_metrics(val_loader)  
    print(f'Epoch: {epoch}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')
    wandb.log({
        "train_acc": train_acc,
        "train_f1": train_f1,
        "train_auc": train_auc,
        "train_loss": train_loss,
        "train_mcc": train_mcc, 
        "val_acc": val_acc,
        "val_f1": val_f1,
        "val_auc": val_auc,
        "val_loss": val_loss,
        "val_mcc": val_mcc  
    })

# Final evaluation on test set
test_accuracy, test_f1, test_auc, test_loss, test_mcc = calculate_metrics(test_loader)  
print(f'Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}, Test AUC: {test_auc:.4f}, Test MCC: {test_mcc:.4f}')  
wandb.log({"test_accuracy": test_accuracy, "test_f1": test_f1, "test_auc": test_auc, "test_loss": test_loss, "test_mcc": test_mcc})  

wandb.finish()
# %%
